定义一个能接受可选参数的装饰器



In [1]:
from functools import wraps, partial
import logging

In [2]:
def logged(func=None, *, level=logging.DEBUG, name=None, message=None):
    if func is None:
        return partial(logged, level=level, name=name, message=message)
    logname = name if name else func.__module__
    log = logging.getLogger(logname)
    logmsg = message if message else func.__name__
    
    @wraps(func)
    def wrapper(*args, **kwargs):
        log.log(level, logmsg)
        return func(*args, **kwargs)
    return wrapper

# example
@logged
def add(x, y):
    return x + y
@logged(level=logging.CRITICAL, name='example')
def spam():
    print('Spam!')

In [3]:
add(3,4)

7

In [4]:
spam()

spam


Spam!


利用装饰器对函数参数强制执行类型检查

In [6]:
from inspect import signature
from functools import wraps

In [12]:
def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        if not __debug__:
            return func
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be {}'.format(name, bound_types[name]))
            return func(*args, **kwargs)
        return wrapper
    return decorate
                            
                            
                            

In [13]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)

In [21]:
spam(1,'hello',2)

1 hello 2
